In [1]:
# project: p10
# submitter: bai59
# partner: pli233
# hours: 2

In [2]:
import os
import csv
import json
from collections import namedtuple
# you may use from style of import to import JSONDecodeError

In [3]:
def list_files_in(pathname):
    res = []
    files = os.listdir(pathname)
    files.sort()
    for i in files:
        if(i[0]!='.'):
            res.append(i)
    sorted(res)
    return res

In [4]:
#q1
list_files_in("data")

['channel_ids1.json',
 'channel_ids2.json',
 'channel_ids3.json',
 'channel_ids4.json',
 'channel_ids5.json',
 'comment_data1.csv',
 'comment_data2.csv',
 'comment_data3.csv',
 'comment_data4.csv',
 'comment_data5.csv',
 'video_data.csv',
 'video_ids.json']

In [5]:
def list_paths_in(pathname):
    return [os.path.join(pathname,i) for i in list_files_in(pathname)]

In [6]:
#q2
list_paths_in_data = list_paths_in("data")
list_paths_in_data

['data\\channel_ids1.json',
 'data\\channel_ids2.json',
 'data\\channel_ids3.json',
 'data\\channel_ids4.json',
 'data\\channel_ids5.json',
 'data\\comment_data1.csv',
 'data\\comment_data2.csv',
 'data\\comment_data3.csv',
 'data\\comment_data4.csv',
 'data\\comment_data5.csv',
 'data\\video_data.csv',
 'data\\video_ids.json']

In [7]:
#q3
comment_paths = [i for i in list_paths_in_data if 'comment_data' in i]
comment_paths.sort()
comment_paths

['data\\comment_data1.csv',
 'data\\comment_data2.csv',
 'data\\comment_data3.csv',
 'data\\comment_data4.csv',
 'data\\comment_data5.csv']

In [8]:
#q4
channel_paths = [i for i in list_paths_in_data if 'channel_ids' in i]
channel_paths.sort()
channel_paths

['data\\channel_ids1.json',
 'data\\channel_ids2.json',
 'data\\channel_ids3.json',
 'data\\channel_ids4.json',
 'data\\channel_ids5.json']

In [9]:
def read_json(path):
    with open(path, encoding="utf-8") as f:
        return json.load(f) # dict, list, etc
def get_mapping(pathname):
    try:
        return read_json(pathname)
    except json.JSONDecodeError:
        return {}

In [10]:
#q5
get_mapping(os.path.join("data", "channel_ids1.json"))['UCNye-wNBqNL5ZzHSJj3l8Bg']

'Al Jazeera English'

In [11]:
#q6
channel_dict = {}
counter = 0
for i in channel_paths:
    for j in get_mapping(i).items():
        channel_dict[j[0]] = j[1]
        counter += 1
counter

372

In [12]:
Comment = namedtuple("Comment",["video_id","comment_length","author_id","likes","published_at"])

In [13]:
# You will need process_csv in order to complete get_comment_data.
# If you already copy/pasted it from lab, please ignore this copy.
def process_csv(filename):
    exampleFile = open(filename, encoding="utf-8")  
    exampleReader = csv.reader(exampleFile)
    exampleData = list(exampleReader)        
    exampleFile.close()  
    return exampleData

def get_comment_data(comment_file):
    csv_data = process_csv(comment_file)
    header = csv_data[0]
    comment_rows = csv_data[1:]
    comment_id_idx = header.index("comment_id")
    video_id_idx = header.index("video_id")
    comment_length_idx = header.index("comment_length")
    author_id_idx = header.index("author_id")
    likes_idx = header.index("likes")
    published_at_idx = header.index("published_at")
    # Iterate over comment_rows
    res = {}
    for i in comment_rows:
        if len(i) != 6 or '' in i:
            continue
        try:
            res[i[comment_id_idx]] = Comment(i[video_id_idx],int(i[comment_length_idx]),i[author_id_idx],int(i[likes_idx]),i[published_at_idx])
        except ValueError:
            continue
    return res

In [14]:
#q7
#What is the Comment object with comment ID UgygOezB4Mvd5o6FgAt4AaABAg
get_comment_data(os.path.join("data", "comment_data1.csv"))['UgygOezB4Mvd5o6FgAt4AaABAg']

Comment(video_id='udNXMAflbU8', comment_length=175, author_id='UCHkk7x38KWgqjQOHqsQwf0Q', likes=47, published_at='2021-10-10 17:48:38')

In [15]:
comments = {}
for file in comment_paths:
    comments.update(get_comment_data(file))

In [16]:
#q8
#What is the length of the comment with ID UgztIaGfqFoiGvbOdfp4AaABAg
comments['UgztIaGfqFoiGvbOdfp4AaABAg'].comment_length

67

In [17]:
#q9
#What percentage of comments are at most 140 characters long?
len([i for i in comments.values() if i.comment_length<=140])/len(comments) * 100

86.86953042956443

In [18]:
#q10
#What is the author ID of the comment that has the highest number of likes?
max_val = -1
for i in comments.values():
    if i.likes>max_val:
        max_val = i.likes
        max_id = i.author_id
max_id

'UCIPPMRA040LQr5QPyJEbmXA'

In [19]:
#Q11
#calculate the most active hour for posting comment
# dict of 24 hours
hour_dict = {i:0 for i in range(24)}
for i in comments.values():
    hour_dict[int(i.published_at[11:13])] += 1
sorted(hour_dict.items(), key=lambda x: x[1], reverse=True)[0][0]

19

In [20]:
# Bucketize the comments data by creating a dict mapping video IDs to a list of comment IDs corresponding to that video ID.
comment_buckets = {}
for i in comments:
    if comments[i].video_id not in comment_buckets:
        comment_buckets[comments[i].video_id] = [i]
    else:
        comment_buckets[comments[i].video_id].append(i)

In [21]:
#q12
# How many comments does the video with ID A8rrr_w8rfk have?
len(comment_buckets['A8rrr_w8rfk'])

606

In [22]:
def get_videos(data_file, video_mapping_file):
    """
    Given data_file (csv file) and video_mapping_file (json) file, generates a video
    dictionary, mapping video ID to a dictionary containing title, and other details of the video.
    Handles missing entry in video_ids.json by using try / except blocks to handle KeyError.
    """
    data = process_csv(data_file)
    header = data[0]
    all_videos = data[1:]
    video_mapping = get_mapping(video_mapping_file)
    videos_dict = dict()
    for video in all_videos: # You may find it helpful to do all_videos[:5] to only look at the first 5 videos.
        try:
            videos_dict[video[header.index("video_id")]] = {
                "title": video_mapping[video[header.index("video_id")]],
                "ratings_enabled": video[header.index("likes")]!='' and video[header.index("dislikes")]!='',
                "channel_name": channel_dict[video[header.index("channel_id")]],
                "published_at": video[header.index("published_at")],
                "duration": video[header.index("duration")],
                "category": video[header.index("category")],
                "tags": video[header.index("tags")].split("|"),
                "comments": comment_buckets[video[header.index("video_id")]],
                "views": int(video[header.index("views")])
            }
            if videos_dict[video[header.index("video_id")]]["ratings_enabled"]:
                videos_dict[video[header.index("video_id")]]["likes"] =  int(video[header.index("likes")])
                videos_dict[video[header.index("video_id")]]["dislikes"] =  int(video[header.index("dislikes")])
        except KeyError:
            continue
        except ValueError:
            continue
    return videos_dict

videos = get_videos(os.path.join('data','video_data.csv'), os.path.join('data','video_ids.json'))

In [23]:
#Q13
#What is the video with ID fkMW60W180E?
videos['fkMW60W180E']

{'title': 'SWAWS | Totally Accurate Battlegrounds',
 'ratings_enabled': True,
 'channel_name': 'TheRussianBadger',
 'published_at': '2021-10-12 19:01:41',
 'duration': '00:18:46',
 'category': 'Gaming',
 'tags': ['tot',
  'totally accurate battlegrounds',
  'tabg',
  'totally accurate battle simulator',
  'totally accurate battlegrounds gameplay',
  'totally accurate battle grounds',
  'tabg gameplay',
  'tabg game',
  'tabs',
  'totally accurate',
  'totally accurate battle royale',
  'battle royale',
  'tabg funny',
  'fortnite',
  'battlegrounds',
  'tabs battle royale',
  'new battle royale',
  'pubg',
  'totally accurate battlegrounds funny',
  'swaws',
  'swaws meme',
  'swaws russian badger',
  'tabg update',
  'tabg win',
  'tabg br',
  'free to play pc games',
  'free to play'],
 'comments': ['UgzgwN2JXxjTN4mR5954AaABAg.9TPxukUd20g9TQLnJi3RFU',
  'Ugw8z3sbkSQQpgINHdl4AaABAg',
  'UgyjZAl0XlNg-gOZ0jR4AaABAg.9TPyCmI6pbl9TQAXc7oVx_',
  'Ugy1W1ldfeXB5VavpEB4AaABAg',
  'Ugz-5ND1_48b

In [24]:
#Q14
#What is the title of the video with ID gF69voHU_ys?
videos['gF69voHU_ys']['title']

'A Mukbanger Ate 1 Gallon Pickles. This Is What Happened To Her Brain.'

In [25]:
#Q15
#Among the videos with more than 1 million views, what is the title of the video with the highest likes to views ratio?
max_ratio = -1
max_id = None
for i in videos.keys():
    if 'likes' in videos[i] and videos[i]['views']>1000000 and videos[i]['likes']/videos[i]['views']>max_ratio:
        max_ratio = videos[i]['likes']/videos[i]['views']
        max_id = videos[i]['title']
max_id

'[In the SOOP BTS ver. Season 2] Official Teaser 2'

In [26]:
#q16
#What is the author ID of the most liked comment under the video titled 'Giving Away My Beard For Charity!'?
target_movie = 'Giving Away My Beard For Charity!'
target_comments = None
for i in videos.items():
    if i[1]['title'] == target_movie:
        target_comments = i[1]['comments']
        break
max_likes = -1
max_comment_author_id = None
for i in target_comments:
    if comments[i].likes>max_likes:
        max_likes = comments[i].likes
        max_comment_author_id = comments[i].author_id
max_comment_author_id

'UCRvcDpyxCVmlLI8Q2TjRpmg'

In [27]:
def bucketize(attribute, videos=videos):
  #TODO: This is very similar to bucketize from the last project
  buckets = {}
  for i in videos.items():
    if attribute == 'video_id':
        if i[1][attribute] not in buckets:
            buckets[i[0]] = [i[0]]
        else:
            buckets[i[0]].append(i[0])
    elif type(i[1][attribute]) is list:
        for j in i[1][attribute]:
            if j not in buckets:
                buckets[j] = [i[0]]
            else:
                buckets[j].append(i[0])
    else:
        if i[1][attribute] not in buckets:
            buckets[i[1][attribute]] = [i[0]]
        else:
            buckets[i[1][attribute]].append(i[0])
  return buckets

In [28]:
#q17
channel_buckets = bucketize("channel_name")
cc_videos = []
for i in channel_buckets['Corridor Crew']:
    cc_videos.append(videos[i]['title'])
cc_videos

['VFX Artists React to SHANG-CHi Bad & Great CGi',
 'VFX Artists React to Bad & Great CGi 54 (ft. SETH ROGEN)']

In [29]:
#q18
#What are the top 5 channels that have the most total comments on their videos?
top_channels = []
for i in channel_buckets.items():
    top_channels.append((i[0], sum([len(comment_buckets[j]) for j in i[1]])))
top_channels = sorted(top_channels, key=lambda x: x[1], reverse=True)[:5]
[i[0] for i in top_channels]

['NFL', 'Minecraft', 'Saturday Night Live', 'SSSniperWolf', 'PBC ON FOX']

In [30]:
#q19
tag_buckets = bucketize("tags")
res = []
for i in tag_buckets.items():
    if 'minecraft' in i[0].lower():
        for i in i[1]:
            if videos[i]['title'] not in res:
                res.append(videos[i]['title'])
res

['Realistic Golem Transforming',
 'I Survived 100 Days as a PANDA in HARDCORE Minecraft!',
 'I Survived 100 Days as an ENDER DRAGON in Minecraft',
 "Minecraft, But There's Custom End City Items...",
 'Building AUTOMATIC FARMS in Minecraft Hardcore (#52)',
 'I made a custom mod in minecraft with your ideas',
 'Minecraft - Last Life #3: Frightening Exploit Tactics',
 'I Trapped 10 YouTubers on One Block',
 'I Survived 100 Days as a FOX in HARDCORE Minecraft!',
 'LAST TO LEAVE THE MYSTERY MACHINE WINS $10,000',
 'Last Life: Episode 4 - UH OH!',
 'Minecraft But I Upgrade EVERYTHING - The ULTIMATE Survival World! | Part 4',
 'Minecraft, But You Can Upgrade TNT...',
 'You Laugh You Lose Is Even Funnier In Real Life...',
 'can 1 pixel of water save you?',
 'Philza And Wilbur TEACH Tommy To FLY And Its SCUFFED LORE! ORIGINS SMP',
 'Realistic Water in Minecraft...',
 'Hermitcraft 8 Episode 14:  BETRAYING BOATEM',
 'I Found the RICHEST Server in Minecraft! *millionaire*',
 "Minecraft, But You Ca

In [31]:
#Q20
#List the titles of the 5 shortest videos
shortest_videos = []
for i in videos.items():
    shortest_videos.append((i[1]['title'], i[1]['duration']))
[i[0] for i in sorted(shortest_videos, key=lambda x: x[1])[:5]]

['When someone walks in while you using the bathroom😭',
 'Smart Appliances, Gadgets For Every Home 🤩 Versatile Utensils, Kitchen, Makeup, Beauty #Shorts',
 'Italians reacting to ketchup on pasta 😱🇮🇹 #shorts',
 'My hidden talent #shorts',
 'Knot']